<a href="https://colab.research.google.com/github/ShahedSabab/maskOrNot/blob/master/maskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import re

# Defining Model

In [7]:
num_classes = 2
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(num_classes, activation='softmax'))

# not using the first layer for training
model.layers[0].trainable = False

# Compile Model

In [8]:
model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

# Image augmentation for preprocessing image

In [11]:
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True, 
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Take image directly from director and apply augmentation

In [12]:
image_size = 224

train_generator = data_generator_with_aug.flow_from_directory(
                                        directory='/Users/sherapgyaltsen/Documents/Face-mask-detection/train',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')
validation_generator = data_generator_no_aug.flow_from_directory(
                                        directory='/Users/sherapgyaltsen/Documents/Face-mask-detection/val',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

Found 73 images belonging to 2 classes.
Found 34 images belonging to 2 classes.


# Fit the model

In [13]:
fit_stats = model.fit_generator(train_generator,
                                       steps_per_epoch=60,
                                       epochs = 10,
                                       validation_data=validation_generator,
                                       validation_steps=1)

/Users/sherapgyaltsen/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-21 20:43:06.484921: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/10
60/60 [==============================] - 15s 209ms/step - loss: 1.0212 - accuracy: 0.7237 - val_loss: 0.0292 - val_accuracy: 1.0000


# Testing with test image

In [14]:
#collect test data from directory and process
test_generator = data_generator_no_aug.flow_from_directory(
    directory='/Users/sherapgyaltsen/Documents/Face-mask-detection/test',
    target_size=(image_size, image_size),
    batch_size= 10,
    class_mode=None,
    shuffle=False
)

# Predict from generator (returns probabilities)
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=test_generator.filenames
# Data frame

real_class = []
for file in filenames:
  if re.search("with_mask", file):
    real_class.append(1.0)
  else:
    real_class.append(0.0)

predicted_class = cl[:,0]

results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "pred_class":predicted_class, "real_class":real_class})

Found 30 images belonging to 2 classes.


/Users/sherapgyaltsen/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


3/3 [==============================] - 4s 953ms/step


In [15]:
results

,file,pr,pred_class,real_class
0,with_mask/With_Mask_4.png,0.876198,1.0,1.0
1,with_mask/With_Mask_5.png,0.983058,1.0,1.0
2,with_mask/With_Mask_6.png,0.817192,1.0,1.0
3,with_mask/With_Mask_7.png,0.771160,1.0,1.0
4,with_mask/With_Mask_8.png,0.564043,1.0,1.0
5,with_mask/With_Mask_9.png,0.404292,0.0,1.0
6,with_mask/opencv_frame_0.png,0.970470,1.0,1.0
7,with_mask/opencv_frame_1.png,0.983075,1.0,1.0
8,with_mask/opencv_frame_2.png,0.993000,1.0,1.0
9,with_mask/opencv_frame_3.png,0.986384,1.0,1.0


In [16]:
res = sum(1 for x,y in zip(real_class,predicted_class) if x == y) / len(real_class)
print("Accuracy :", res)

Accuracy : 0.9333333333333333


# Save Model

In [17]:
model.save('maskDetectionModel_2.h5')